# 7.4 문자열 다루기

- pandas는 배열 데이터 전체에 쉽게 정규표현식을 적용하고 추가적으로 누락된 데이터를 편리하게 처리할 수 있는 기능을 포햠하고 있다.

## 7.4.1 문자열 객체 메서드

- 문자열을 다뤄야 하는 대부분의 프로그램은 내장 문자열 메서드만으로 충분하다.

In [8]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [9]:
val = 'a,b,  guido'

In [10]:
val.split(',')

['a', 'b', '  guido']

- split 메서드는 종종 공백문자를 제거하는 strip 메서드와 조합해서 사용하기도 한다.

In [11]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

- 이렇게 분리된 문자열은 더하기 연산을 통해 '::' 문자열과 합칠 수도 있다.

In [12]:
first, second, third = pieces

In [13]:
first +  '::' + second + '::' + third

'a::b::guido'

- 좀 더 빠르고 파이썬스러운 방법은 join을 사용하는 것이다.

In [14]:
'::'.join(pieces)

'a::b::guido'

- 일치하는 부분 문자열의 위치를 찾는 방법
- index, find, in

In [15]:
'guido' in val

True

In [16]:
val.index(',')

# val.index(':') # ValueError

1

In [17]:
val.find(',')

1

In [18]:
val.find(':')

-1

- find와 index의 차이점은 index의 경우 문자열을 찾지 못하면 예외를 발생시킨다는 점이다.
- find는 문자열을 찾지 못하면 -1을 반환한다.

- count는 특정 부분 문자열이 몇 건 발견되었는지를 반환

In [19]:
val.count(',')

2

- replace는 찾아낸 패턴을 다른 문자열로 치환한다.
- 패턴을 삭제하기 위한 방법으로 자주 사용한다.

In [20]:
val.replace(',', '::')

'a::b::  guido'

In [21]:
val.replace(',', '')

'ab  guido'

## 정규표현식

- re 모듈 함수는 패턴 매칭, 치환, 분리의 세 가지로 나눌 수 있다.
- 정규표현식은 텍스트 안에 존재하는 패턴을 표현하고 이를 여러 가지 다양한 목적으로 사용할 수 있도록 되어 있다.

- 여러 가지 공백문자(탭, 스페이스, 개행문자)가 포함된 문자열을 나누고 싶다면 하나 이상의 공백문자를 의미하는 `\s+`를 사용해서 문자열을 분리할 수 있다.

In [22]:
import re

In [23]:
text = "foo     bar\t baz    \tqux"

In [24]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- 먼저 정규표현식ㅇ이 컴파일되고 그 다음에 split 메서드가 실행된다.
- re.compile을 통해 직접 정규표현식을 컴파일하고 그렇게 얻은 정규표현식 객체를 재사용하는 것도 가능하다.

In [25]:
regex = re.compile('\s+')

In [26]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

- 정규표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드를 사용한다.

In [27]:
regex.findall(text)

['     ', '\t ', '    \t']

- 정규표현식 안에서 `\`문자가 이스케이프되는 것을 피하려면 raw 문자열 표기법으로 문제를 회피할 수 있다.
- `r''`

- 같은 정규표현식을 다른 문자열에도 적용해야 한다면 re.compile을 이용해서 정규표현식 객체를 만들어 사용하는 방법을 추천한다.

- match와 search는 findall 메서드와 관련이 있다.
- findall은 문자열에서 일치하는 모든 부분의 문자열을 찾아주지만 search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다.

In [28]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [29]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [30]:
regex = re.compile(pattern, flags=re.IGNORECASE)

- findall 메서드를 사용하면 주어진 텍스트의 모든 이메일 주소를 확인할 수 있다.

In [31]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

- search는 텍스트에서 첫 번째 이메일 주소만 확인할 수 있다.
- 이 정규표현식에 대한 match 객체는 그 패턴이 문자열 안에서 위치하는 시작점과 끝점만을 알려준다.

In [32]:
m = regex.search(text)

In [33]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [34]:
text[m.start():m.end()]

'dave@google.com'

In [35]:
print(regex.match(text)) # 이건 무슨 의미지?

None


- sub 메서드는 찾을 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.

In [36]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



- 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사의 세 가지 컴포넌트로 나누어야 한다면 각 패컨을 괄호로 묶어준다.

In [37]:
pattern = r'([A-Z0-9._%+-]+)@([[A-Z0-9.-]+)\.([A-Z]{2,4})'

In [38]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [39]:
m = regex.match('wesm@bright.net')

In [40]:
m.groups()

('wesm', 'bright', 'net')

- 패턴에 그룹이 있다면 findall 메서드는 튜플의 리스트를 반환한다.

In [41]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

- sub 메서드는 `\1, \2` 같은 특수한 기호룰 사용해서 각 패턴의 그룹에 접근할 수 있다.

In [42]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



- 정규표현식에서 변수명을 줄 수도 있다.

In [43]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [44]:
m = regex.match('wesm@bright.net')

In [45]:
m.groupdict()

{'username': 'wesm', 'domain': 'bright', 'suffix': 'net'}

###  정규표현식 메서드
- findall: 문자열에서 겹치지 않는, 발견된 모든 패턴을 반환한다.
- finditer: 이터레이터를 통해 하나씩 반환한다.
- match: 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다(?)
    - 일치하는 패턴이 있다면 match 객체를 반환, 그렇지 않을 경우 None을 반환

## pandas의 벡터화된 문자열 함수

- 문자열과 정규표현식 메서드는 data.map을 사용해서 각 값에 적용할 수 있지만 NA 값을 만나면 실패한다.
- 이런 문제에 대처하기 위해 Series에는 NA 값을 건너뛰게 하는 간결한 문자열 처리 메서드가 있다.
- 예들 들어 각 이메일 주소가 'gmail'을 포함하고 있는지 str.contains를 이용해 검사할 수 있다.

In [46]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [47]:
data = Series(data)

In [48]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [49]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [50]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

- IGNORECASE 같은 re 옵션과 함께 정규표현식도 사용할 수 있다.

In [51]:
import re
pattern = r'[A-Z0-9._%+-]+@A-Z0-9.-]+\.[A-Z]{2,4}'

In [52]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave      []
Rob       []
Steve     []
Wes      NaN
dtype: object